In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

### Loading the dataset

In [3]:
training_data = datasets.FashionMNIST(
    root="data/mnist",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data/mnist",
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

### Creating the model class

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

### Calling the model

In [6]:
model = NeuralNetwork()

### Hyperparameters

In [7]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Loss

In [8]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### train loop

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        # Forward step
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # reseting gradients
        loss.backward()
        optimizer.step() # adjust gradients

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss}  [{current} / {size}]")

In [12]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\nAccuracy: {100*correct} %, Avg loss: {test_loss}\n")

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.3149092197418213  [0 / 60000]
loss: 2.299694299697876  [6400 / 60000]
loss: 2.2757961750030518  [12800 / 60000]
loss: 2.26698637008667  [19200 / 60000]
loss: 2.2514264583587646  [25600 / 60000]
loss: 2.2296760082244873  [32000 / 60000]
loss: 2.23718523979187  [38400 / 60000]
loss: 2.206199884414673  [44800 / 60000]
loss: 2.209331750869751  [51200 / 60000]
loss: 2.1728098392486572  [57600 / 60000]
Test Error:
Accuracy: 38.65 %, Avg loss: 2.164371417586211

Epoch 2
-------------------------------
loss: 2.1809310913085938  [0 / 60000]
loss: 2.1716196537017822  [6400 / 60000]
loss: 2.1121816635131836  [12800 / 60000]
loss: 2.12983775138855  [19200 / 60000]
loss: 2.08646821975708  [25600 / 60000]
loss: 2.027327299118042  [32000 / 60000]
loss: 2.058231830596924  [38400 / 60000]
loss: 1.980369210243225  [44800 / 60000]
loss: 1.9903041124343872  [51200 / 60000]
loss: 1.926987648010254  [57600 / 60000]
Test Error:
Accuracy: 51.04 %, Avg loss: 1.91